# `2` European Options
A European option is the type of options contract that allows investors to exercise their options only on the expiration date of that contract.

run `fetch_data.py` to fetch data
<br />
if the data already fetched (`data_old` and `data_new` already available in `data` folder), skip this step

`2.1` Import all the dependencies

In [17]:
from nsepython import *
from datetime import date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

`2.2` Set `inflation rate` and `bond yield` to calculate `risk free rate` of return for our model
![./images/rfr.png](./images/rfr.png)

In [18]:
# Inflation rate is 3.85% as of 2023 April
inflation_rate = 0.0385

# 10 year bond yield is 7.229% as of 2023 April
bond_yield = 0.07229

# Risk free rate calculation
risk_free_rate = ((1 + bond_yield) /  (1 + inflation_rate)) - 1

print("Risk free rate is: ", risk_free_rate)

Risk free rate is:  0.032537313432835724


`2.3` As the required data is already fetced, read the csv files(`data_old` and `data_new`) from `data` folder and store the data in respective dataframes
<br />

In [19]:

data_old = pd.read_csv('./data/data_old.csv')
data_new = pd.read_csv('./data/data_new.csv')

`2.4` Add new colunm `log_returns` to the dataframe to store log returns
<br />
The log returns are calculated by taking the difference between the log of the `closing price of the current day` and the log of the `closing price of the previous day`.
<br />
`d_std` is the standard deviation of the daily returns
<br />
calculate `annualized_volatility` by multiplying the `daily standard deviation` by the square root of `252` (number of trading days in a year)
<br />
![./images/daily_returns.png](./images/volatility.png)
<br />
`spot_price` is the closing price of the stock on the last trading day of the year

In [20]:
data_old['log_returns'] = np.log(data_old['CH_CLOSING_PRICE'] / data_old['CH_CLOSING_PRICE'].shift(1))
# export to csv
data_old.to_csv(r'./data/data_old_log_returns.csv', index=False)

d_std = data_old['log_returns'].std()
annualized_volatility = d_std * 250 ** 0.5

# Spot price
spot_price = data_old['CH_CLOSING_PRICE'].iloc[-1]

In [21]:
x = data_old["CH_CLOSING_PRICE"].values
n = x.shape[0]
window = int(10*252)
# Calculating the log returns
log_returns = np.log(x[n-window:n-1]/x[n-window-1:n-2])

# Calculating the annualized volatility
annualized_volatility = np.std(log_returns) * np.sqrt(252)
spot_price = x[n-1]

In [22]:
print(annualized_volatility)
print(spot_price)

0.6004573547535781
1071.15


In [23]:
m=10000
T=30.0/252
# for t in range(T):
def generate_prices(spot_price, risk_free_rate, annualized_volatility, T):
    expected_price = spot_price * np.exp((risk_free_rate - 0.5 * annualized_volatility ** 2) * T + annualized_volatility * np.sqrt(T) * np.random.normal(0, 1, m))
    return expected_price
mean_price = generate_prices(spot_price, risk_free_rate, annualized_volatility, T)
print(spot_price)
print(mean_price)

1071.15
[ 745.07367825 1130.50918951  783.88979327 ... 1274.17853198 1275.99742678
 1069.16950802]


In [24]:
strike_price = 1090.00
# spot_price = 168.34
# annualized_volatility = 0.593305
# risk_free_rate = 0.0202
# T = 0.147945

# call_payoff = np.maximum(mean_price - strike_price, 0)
# put_payoff = np.maximum(strike_price - mean_price, 0)

# print(call_payoff)
# print(put_payoff)

In [25]:
# Calculating Payoffs
def payoff_generation(strike_price, simulation_count = 10000):
    call_payoffs = []
    put_payoffs = []
    for i in range(simulation_count):
        mean_price = generate_prices(spot_price, risk_free_rate, annualized_volatility, T)
        call_payoff = np.maximum(mean_price - strike_price, 0)
        put_payoff = np.maximum(strike_price - mean_price, 0)
        call_payoffs.append(call_payoff)
        put_payoffs.append(put_payoff)
    return call_payoffs, put_payoffs

In [26]:
# Calculating the call and put option prices
def option_price_calculation(strike_price, simulation_count = 10000):
    call_payoffs, put_payoffs = payoff_generation(strike_price, simulation_count)
    call_price = np.mean(call_payoffs) * np.exp(-risk_free_rate * T)
    put_price = np.mean(put_payoffs) * np.exp(-risk_free_rate * T)
    return call_payoffs, put_payoffs, call_price, put_price

In [27]:
# Executing the function
call_payoffs, put_payoffs, call_price, put_price = option_price_calculation(strike_price)

# Printing the results
print("Call Option price is: ", call_price)
print("Put Option price is: ", put_price)

Call Option price is:  81.84386433006144
Put Option price is:  96.50304000573448


In [28]:
# Calculating Put Call Parity
def put_call_parity_calculation(call_price, put_price, spot_price, strike_price, risk_free_rate, T):
    lhs = call_price - put_price
    rhs = spot_price - strike_price * np.exp(-risk_free_rate * T)
    return round(lhs) == round(rhs)

# Executing the function
put_call_parity_calculation(call_price, put_price, spot_price, strike_price, risk_free_rate, T)

True

In [29]:
# Now lets calculate the option price using Black Scholes formula
# First we need to calculate the d1 and d2 values

# Calculating d1
def d1_calculation(spot_price, strike_price, risk_free_rate, annualized_volatility, T):
    d1 = (np.log(spot_price/strike_price) + (risk_free_rate + 0.5 * annualized_volatility**2) * T) / (annualized_volatility * np.sqrt(T))
    return d1

# Calculating d2
def d2_calculation(spot_price, strike_price, risk_free_rate, annualized_volatility, T):
    d2 = (np.log(spot_price/strike_price) + (risk_free_rate - 0.5 * annualized_volatility**2) * T) / (annualized_volatility * np.sqrt(T))

In [30]:
# Calculating the call and put option prices using Black Scholes formula
def option_price_calculation_black_scholes(spot_price, strike_price, risk_free_rate, annualized_volatility, T):
    d1 = d1_calculation(spot_price, strike_price, risk_free_rate, annualized_volatility, T)
    d2 = d2_calculation(spot_price, strike_price, risk_free_rate, annualized_volatility, T)
    call_price = spot_price * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * T) * norm.cdf(d2)
    put_price = strike_price * np.exp(-risk_free_rate * T) * norm.cdf(-d2) - spot_price * norm.cdf(-d1)
    return call_price, put_price

In [31]:
# Executing the function
call_price_black_scholes, put_price_black_scholes = option_price_calculation_black_scholes(spot_price, strike_price, risk_free_rate, annualized_volatility, T)
print("Call Option price using Black Scholes formula is: ", call_price_black_scholes)
print("Put Option price using Black Scholes formula is: ", put_price_black_scholes)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'